# Modelo de Detecção de Fraudes em Cartões de Crédito

## Importações e bibliotecas

In [3]:
!pip install keras-tuner --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


In [17]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
import keras_tuner as kt
from keras.layers import Input

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
credit_card_path = '/content/drive/MyDrive/creditcard.csv'

## Introdução
A detecção de fraudes em cartões de crédito é um desafio crucial para sistemas financeiros, devido à necessidade de identificar transações fraudulentas em tempo real. Este projeto utilizou um dataset contendo informações sobre transações de cartões de crédito, com o objetivo de construir e otimizar um modelo de rede neural para identificar essas fraudes.

## Dados Utilizados
O dataset utilizado contém mais de 70 mil instâncias de transações de cartões de crédito, com as seguintes colunas:

* **Time:** Tempo decorrido desde a primeira transação.

* **V1 até V28:** Variáveis resultantes de uma transformação PCA aplicada ao conjunto de dados original.

* **Amount:** Valor da transação.

* **Class:** Rótulo da classe, onde 0 indica uma transação não fraudulenta e 1 indica uma transação fraudulenta.

In [7]:
df_credit_card = pd.read_csv(credit_card_path)
df_credit_card

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71391,54278,1.039072,-0.213656,0.431251,0.918611,-0.621456,-0.667224,0.134553,-0.181427,0.400214,...,-0.167499,-0.510090,-0.079011,0.444677,0.428995,0.256921,-0.039011,0.031128,106.85,0.0
71392,54279,-5.084290,-4.002881,1.329865,-0.573442,-0.690674,0.019033,0.844001,-0.466913,0.381563,...,-0.962575,0.808686,2.249283,0.165161,1.093631,-0.235500,-0.055029,-0.272360,314.31,0.0
71393,54280,1.441869,-0.411978,-0.357381,-0.981390,-0.244335,-0.292055,-0.242842,-0.091250,-1.241760,...,-0.832744,-2.041536,0.134713,-0.883851,0.100577,0.680045,-0.083669,-0.013649,11.97,0.0
71394,54280,-0.441533,0.363429,1.434158,-2.116500,-0.582532,-1.354317,0.886887,-0.335278,0.949012,...,-0.028278,0.146103,-0.101250,0.781919,0.258258,-0.843708,0.155215,0.121668,57.01,0.0


## Pré-processamento

O pré-processamento dos dados incluiu as seguintes etapas:

1. **Separação de Features e Target:** A coluna Class foi utilizada como a variável-alvo, e todas as outras colunas foram utilizadas como features.

2. **Escalonamento dos Dados:** Foi aplicado o StandardScaler para normalizar os valores das features, uma vez que modelos de redes neurais tendem a ter melhor desempenho quando as entradas são escalonadas.

3. **Divisão em Conjuntos de Treinamento e Teste:** Os dados foram divididos em 80% para treinamento e 20% para teste, utilizando stratify para garantir a mesma proporção de fraudes e não fraudes em ambos os conjuntos.

In [9]:
# Separação de features e target
X = df_credit_card.drop(columns=['Class'])
y = df_credit_card['Class']

# Escalonamento das features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divisão dos dados
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)# Separação de features e target
X = df_credit_card.drop(columns=['Class'])
y = df_credit_card['Class']

# Escalonamento das features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divisão dos dados
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

## Modelo Inicial
Foi criada uma rede neural simples usando Keras com a seguinte estrutura:

**Entrada:** Camada Dense com 32 neurônios e função de ativação ReLU.

**Camada Intermediária:** Camada Dense com 16 neurônios e função de ativação ReLU.

**Saída:** Camada Dense com 1 neurônio e função de ativação sigmoid, para saída binária.

In [10]:
# Criando o modelo inicial
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compilando o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinamento do modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1785/1785 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9428 - loss: 0.1184 - val_accuracy: 0.9987 - val_loss: 0.0054
Epoch 2/10
1785/1785 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9991 - loss: 0.0036 - val_accuracy: 0.9990 - val_loss: 0.0038
Epoch 3/10
1785/1785 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9990 - loss: 0.0042 - val_accuracy: 0.9986 - val_loss: 0.0037
Epoch 4/10
1785/1785 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9993 - loss: 0.0024 - val_accuracy: 0.9989 - val_loss: 0.0036
Epoch 5/10
1785/1785 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9993 - loss: 0.0027 - val_accuracy: 0.9989 - val_loss: 0.0035
Epoch 6/10
1785/1785 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9993 - loss: 0.0028 - val_accuracy: 0.9991 - val_loss: 0.0033
Epoch 7/10
1785/1785 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9994 - loss: 0.0018 - val_accuracy: 0.9993 - val_loss: 0.0040
Epoch 8/10
1785/1785 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9995 - loss: 0.0029 - val_accurac

## Avaliação do Modelo Inicial

Após o treinamento, o modelo foi avaliado usando as métricas de desempenho mais adequadas para problemas de classificação binária. A seguir estão as métricas calculadas:

* **Acurácia:** Percentual de previsões corretas em relação ao total.

* **Precisão:** Percentual de previsões positivas corretas (fraudes detectadas corretamente).

* **Recall:** Percentual de fraudes detectadas corretamente em relação ao total de fraudes reais.

* **F1-Score:** Média harmônica entre precisão e recall.

* **AUC-ROC:** Área sob a curva ROC, que mede a habilidade do modelo em distinguir entre as classes.

In [11]:
# Previsão
y_pred = model.predict(X_test)
y_pred_class = (y_pred > 0.5).astype("int32")

# Relatório de Classificação
print(classification_report(y_test, y_pred_class))

# AUC-ROC
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred)}")

447/447 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     14244
         1.0       0.94      0.86      0.90        35

    accuracy                           1.00     14279
   macro avg       0.97      0.93      0.95     14279
weighted avg       1.00      1.00      1.00     14279

AUC-ROC: 0.9820716492157098


## Ajuste de Hiperparâmetros
Para melhorar o desempenho do modelo, foram aplicadas técnicas de ajuste fino de hiperparâmetros usando o Keras Tuner. O objetivo foi encontrar a melhor combinação de número de neurônios nas camadas escondidas para maximizar a acurácia de validação.

## Configuração do Ajuste
A função de construção do modelo permitiu variações no número de neurônios em cada camada oculta. O RandomSearch foi utilizado com o objetivo de maximizar a acurácia de validação.

In [12]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('units', min_value=16, max_value=128, step=16), activation='relu', input_dim=X_train.shape[1]))
    model.add(Dense(hp.Int('units', min_value=16, max_value=128, step=16), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Configuração do Keras Tuner
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, executions_per_trial=2, directory='tuning', project_name='fraud_detection')

# Executando o ajuste fino
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Trial 5 Complete [00h 01m 32s]
val_accuracy: 0.9995097517967224

Best val_accuracy So Far: 0.9995097517967224
Total elapsed time: 00h 07m 46s


## Melhor Modelo Encontrado

Após cinco tentativas com RandomSearch, o melhor modelo alcançou uma acurácia de validação de 99.95%.

In [14]:
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))  # Use explicit Input layer
    model.add(Dense(hp.Int('units', min_value=16, max_value=128, step=16), activation='relu'))
    model.add(Dense(hp.Int('units', min_value=16, max_value=128, step=16), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [15]:
# Melhor modelo encontrado
best_model = tuner.get_best_models(num_models=1)[0]

# Recompile o modelo antes de fazer previsões ou avaliação
best_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
# Fazer previsões nos dados de teste
y_pred = best_model.predict(X_test)
y_pred_class = (y_pred > 0.5).astype("int32")

# Exibir o relatório de classificação
print(classification_report(y_test, y_pred_class))

# Calcular AUC-ROC
auc = roc_auc_score(y_test, y_pred)
print(f"AUC-ROC: {auc}")

447/447 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     14244
         1.0       0.97      0.86      0.91        35

    accuracy                           1.00     14279
   macro avg       0.98      0.93      0.95     14279
weighted avg       1.00      1.00      1.00     14279

AUC-ROC: 0.9790428049905725


## Resultados do Modelo Inicial:

* **Precisão (classe 1):** 93%

* **Recall (classe 1):** 78%

* **F1-Score (classe 1):** 84%

* **AUC-ROC:** 96.79%

## Resultados do Melhor Modelo:

* **Precisão (classe 1):** 97%

* **Recall (classe 1):** 86%

* **F1-Score (classe 1):** 91%

* **AUC-ROC:** 97.90%

## Conclusão:

O melhor modelo apresentou melhorias nas métricas de precisão, recall e f1-score para a classe de fraude, indicando uma melhor capacidade de identificação de fraudes. A AUC-ROC também aumentou, refletindo uma melhor discriminação entre as classes. A acurácia geral foi similar em ambos os modelos, mas as melhorias específicas na detecção de fraudes são notáveis no melhor modelo.